In [1]:
pip install -U pandasql

Requirement already up-to-date: pandasql in c:\users\crie072\anaconda3\lib\site-packages (0.7.3)
Note: you may need to restart the kernel to use updated packages.


pip install -U xlwt

pip install -U openpyxl

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import csv
import xlrd
import io
from zipfile import ZipFile
from datetime import date, timedelta, time, datetime

from pandasql import sqldf

from pandas.tseries.offsets import MonthEnd

In [3]:
today_yyyymmdd = date.today().strftime("%Y%m%d")
print(today_yyyymmdd)

20200903


In [4]:
#pandasの表示列数を増やす
import pandas as pd
pd.set_option('display.max_columns', 100)

In [5]:
data_dir = Path(r"C:\Users\crie072\Documents\01_データ関連\Adhoc\202008_退会顧客")

#input
input_file = "CANCEL_CUST"
input_file2 = "cancel_code"

#out_put


退会顧客データデータの読込＆集計

In [6]:
df = pd.read_csv(data_dir /f"{input_file}.csv", encoding='shift=jis')
df.head()

,cp_date,daihyouno,serialno,card,brand,channel,cancel_date,cancel_yymm,last_use_date,kzk_f,birthday,age,gender,income,income2,prefecture,job,occupation,residense,email_f,web_f,cancel_code,kanyu_date,membership_term,shopping_amt_3month,shopping_amt_6month,shopping_amt_12month,nenkaihi,kirikae_date
0,20200731,155202803245,4980006001111007,ゴールド,Visa,郵送,20200215,202002,0,0,19371210,80歳以上,男性,1000,11_1000万円以上,奈良県,0,0,A,0,0,A,20070514,10年以上,0,0,7,200707,0
1,20200731,155205651583,4980006001308009,ゴールド,Visa,郵送,20190523,201905,20181226,0,19361102,80歳以上,男性,0,00_0円,京都府,1,0,9,0,0,A,20070514,10年以上,0,0,0,200707,0
2,20200731,155201966217,4980006003030007,ゴールド,Visa,郵送,20190909,201909,0,0,19440130,75〜79歳,男性,1000,11_1000万円以上,大阪府,3,0,9,0,0,B,20070518,10年以上,0,0,0,200707,0
3,20200731,155202465250,4980006003423004,ゴールド,Visa,郵送,20191202,201912,20191127,0,19590718,60〜64歳,男性,1750,11_1000万円以上,大阪府,0,0,0,0,0,A,20070518,10年以上,0,0,2,200707,0
4,20200731,155202816262,4980006003768002,ゴールド,Visa,郵送,20200408,202004,20191128,0,19450603,75〜79歳,男性,1000,11_1000万円以上,大阪府,3,0,9,0,0,A,20070518,10年以上,0,0,9,200707,0


In [7]:
print(df['kirikae_date'].unique())

[       0 20190624 20200228 ... 20110823 20200427 20110527]


_df = df.copy()
_df.head()

In [8]:
#日付無しデータをダミーの未来日付設定
for i in range(len(df)):
    if df['last_use_date'][i] == 0:
        df.loc[:,'last_use_date'][i] = 20501231#最終利用日

for i in range(len(df)):
    if df['kirikae_date'][i] == 0:
        df.loc[:,'kirikae_date'][i] = 20501231#切替日

C:\Users\crie072\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\crie072\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [9]:
df.head()

,cp_date,daihyouno,serialno,card,brand,channel,cancel_date,cancel_yymm,last_use_date,kzk_f,birthday,age,gender,income,income2,prefecture,job,occupation,residense,email_f,web_f,cancel_code,kanyu_date,membership_term,shopping_amt_3month,shopping_amt_6month,shopping_amt_12month,nenkaihi,kirikae_date
0,20200731,155202803245,4980006001111007,ゴールド,Visa,郵送,20200215,202002,20501231,0,19371210,80歳以上,男性,1000,11_1000万円以上,奈良県,0,0,A,0,0,A,20070514,10年以上,0,0,7,200707,20501231
1,20200731,155205651583,4980006001308009,ゴールド,Visa,郵送,20190523,201905,20181226,0,19361102,80歳以上,男性,0,00_0円,京都府,1,0,9,0,0,A,20070514,10年以上,0,0,0,200707,20501231
2,20200731,155201966217,4980006003030007,ゴールド,Visa,郵送,20190909,201909,20501231,0,19440130,75〜79歳,男性,1000,11_1000万円以上,大阪府,3,0,9,0,0,B,20070518,10年以上,0,0,0,200707,20501231
3,20200731,155202465250,4980006003423004,ゴールド,Visa,郵送,20191202,201912,20191127,0,19590718,60〜64歳,男性,1750,11_1000万円以上,大阪府,0,0,0,0,0,A,20070518,10年以上,0,0,2,200707,20501231
4,20200731,155202816262,4980006003768002,ゴールド,Visa,郵送,20200408,202004,20191128,0,19450603,75〜79歳,男性,1000,11_1000万円以上,大阪府,3,0,9,0,0,A,20070518,10年以上,0,0,9,200707,20501231


In [57]:
#各種日付データの変換
df['cp']=pd.to_datetime(df['cp'], format='%Y%m%d')#基準月
df['kanyu_date']=pd.to_datetime(df['kanyu_date'], format='%Y%m%d')#加入日
df['nenkaihi'] = pd.to_datetime(df['nenkaihi'], format="%Y%m") + MonthEnd(1)#年会費
df['last_use_date']=pd.to_datetime(df['last_use_date'], format='%Y%m%d')#最終利用日
df['kirikae_date']=pd.to_datetime(df['kirikae_date'], format='%Y%m%d')#切替日
df.head()

,cp,no,card,brand,channel,last_use_date,kzk_f,age,gender,email_f,web_f,cancel_code,kanyu_date,membership_term,shopping_amt_6month,nenkaihi,kirikae_date
0,2019-03-31,4980006005584,ゴールド,Visa,郵送,2018-08-11,0,80歳以上,女性,0,0,A,2007-05-18,10年以上,0,2007-07-31,2050-12-31
1,2019-03-31,4980006016405,ゴールド,Visa,郵送,2019-02-20,0,55〜59歳,女性,0,0,A,2007-06-11,10年以上,2,2007-08-31,2050-12-31
2,2019-03-31,4980006018771,ゴールド,Visa,郵送,2050-12-31,0,80歳以上,男性,0,0,A,2007-06-20,10年以上,0,2007-08-31,2050-12-31
3,2019-03-31,4980006020224,ゴールド,Visa,郵送,2008-07-08,0,75〜79歳,男性,0,0,A,2007-06-20,10年以上,0,2007-08-31,2050-12-31
4,2019-03-31,4980006031967,ゴールド,Visa,郵送,2017-12-13,0,60〜64歳,男性,0,0,A,2008-02-04,10年以上,0,2008-04-30,2050-12-31


In [10]:
_df = df.copy()

In [68]:
#経過月数(最終利用日と基準月)
df['mob_last_use']=(df['cp'].dt.year - df['last_use_date'].dt.year) * 12 + df['cp'].dt.month - df['last_use_date'].dt.month

#経過月数(年会費徴取月と基準月)
df['dif_nenkaihi']= df['cp'].dt.month - df['nenkaihi'].dt.month
df.head()

,cp,no,card,brand,channel,last_use_date,kzk_f,age,gender,email_f,web_f,cancel_code,kanyu_date,membership_term,shopping_amt_6month,nenkaihi,kirikae_date,mob_last_use,dif_nenkaihi
0,2019-03-31,4980006005584,ゴールド,Visa,郵送,2018-08-11,0,80歳以上,女性,0,0,A,2007-05-18,10年以上,0,2007-07-31,2050-12-31,7,-4
1,2019-03-31,4980006016405,ゴールド,Visa,郵送,2019-02-20,0,55〜59歳,女性,0,0,A,2007-06-11,10年以上,2,2007-08-31,2050-12-31,1,-5
2,2019-03-31,4980006018771,ゴールド,Visa,郵送,2050-12-31,0,80歳以上,男性,0,0,A,2007-06-20,10年以上,0,2007-08-31,2050-12-31,-381,-5
3,2019-03-31,4980006020224,ゴールド,Visa,郵送,2008-07-08,0,75〜79歳,男性,0,0,A,2007-06-20,10年以上,0,2007-08-31,2050-12-31,128,-5
4,2019-03-31,4980006031967,ゴールド,Visa,郵送,2017-12-13,0,60〜64歳,男性,0,0,A,2008-02-04,10年以上,0,2008-04-30,2050-12-31,15,-1


In [70]:
print(df['cancel_code'].unique())

['A' '2' 'Q' '4' 'X' 'C' 'N' 'W' 'I' 'B' 'K' 'P' 'H' 'D' 'F' 'J' 'V' 'G'
 'Y' 'O' 'E' 'L' 'M' '1']


In [91]:
df_code = pd.read_csv(data_dir /f"{input_file2}.csv", encoding='shift=jis')
df_code.head()

,cancel_code,cancel_reason
0,0,通常
1,1,再発行済み
2,2,カード種別変更
3,3,自動カード種別変更
4,4,提携会社コード変更


In [106]:
df_merge = pd.merge(df,df_code, on=['cancel_code'], how= 'left')
df_merge.head()

,cp,no,card,brand,channel,last_use_date,kzk_f,age,gender,email_f,web_f,cancel_code,kanyu_date,membership_term,shopping_amt_6month,nenkaihi,kirikae_date,mob_last_use,dif_nenkaihi,cancel_reason
0,2019-03-31,4980006005584,ゴールド,Visa,郵送,2018-08-11,0,80歳以上,女性,0,0,A,2007-05-18,10年以上,0,2007-07-31,2050-12-31,7,-4,任意解約その他
1,2019-03-31,4980006016405,ゴールド,Visa,郵送,2019-02-20,0,55〜59歳,女性,0,0,A,2007-06-11,10年以上,2,2007-08-31,2050-12-31,1,-5,任意解約その他
2,2019-03-31,4980006018771,ゴールド,Visa,郵送,2050-12-31,0,80歳以上,男性,0,0,A,2007-06-20,10年以上,0,2007-08-31,2050-12-31,-381,-5,任意解約その他
3,2019-03-31,4980006020224,ゴールド,Visa,郵送,2008-07-08,0,75〜79歳,男性,0,0,A,2007-06-20,10年以上,0,2007-08-31,2050-12-31,128,-5,任意解約その他
4,2019-03-31,4980006031967,ゴールド,Visa,郵送,2017-12-13,0,60〜64歳,男性,0,0,A,2008-02-04,10年以上,0,2008-04-30,2050-12-31,15,-1,任意解約その他


In [107]:
#日付データに隠れた時間情報を削除
df_merge['cp'] = df_merge['cp'].dt.strftime('%Y/%m/%d')
df_merge['last_use_date'] = df_merge['last_use_date'].dt.strftime('%Y/%m/%d')
df_merge['kanyu_date'] = df_merge['kanyu_date'].dt.strftime('%Y/%m/%d')
df_merge['kirikae_date'] = df_merge['kirikae_date'].dt.strftime('%Y/%m/%d')
#df_merge['nenkaihi'] = df_merge['nenkaihi'].dt.strftime('%Y/%m/%d')
df_merge['nenkaihi_month'] = df_merge['nenkaihi'].dt.month
df_merge.head()

,cp,no,card,brand,channel,last_use_date,kzk_f,age,gender,email_f,web_f,cancel_code,kanyu_date,membership_term,shopping_amt_6month,nenkaihi,kirikae_date,mob_last_use,dif_nenkaihi,cancel_reason,nenkaihi_month
0,2019/03/31,4980006005584,ゴールド,Visa,郵送,2018/08/11,0,80歳以上,女性,0,0,A,2007/05/18,10年以上,0,2007-07-31,2050/12/31,7,-4,任意解約その他,7
1,2019/03/31,4980006016405,ゴールド,Visa,郵送,2019/02/20,0,55〜59歳,女性,0,0,A,2007/06/11,10年以上,2,2007-08-31,2050/12/31,1,-5,任意解約その他,8
2,2019/03/31,4980006018771,ゴールド,Visa,郵送,2050/12/31,0,80歳以上,男性,0,0,A,2007/06/20,10年以上,0,2007-08-31,2050/12/31,-381,-5,任意解約その他,8
3,2019/03/31,4980006020224,ゴールド,Visa,郵送,2008/07/08,0,75〜79歳,男性,0,0,A,2007/06/20,10年以上,0,2007-08-31,2050/12/31,128,-5,任意解約その他,8
4,2019/03/31,4980006031967,ゴールド,Visa,郵送,2017/12/13,0,60〜64歳,男性,0,0,A,2008/02/04,10年以上,0,2008-04-30,2050/12/31,15,-1,任意解約その他,4


In [96]:
#全体解約数(理由別)
q = """
        SELECT
        cp
        ,card
        ,cancel_code
        ,cancel_reason
        ,count(no) as sum_no_of_account
        from 
            df_merge
        group by
            1,2,3,4
        """
df_all = sqldf(q, locals())
df_all

,cp,card,cancel_code,cancel_reason,sum_no_of_account
0,2019/03/31,ゴールド,2,カード種別変更,8
1,2019/03/31,ゴールド,4,提携会社コード変更,18
2,2019/03/31,ゴールド,A,任意解約その他,187
3,2019/03/31,ゴールド,B,他社カードを使用している,1
4,2019/03/31,ゴールド,C,利用する加盟店がない,1
...,...,...,...,...,...
662,2020/05/31,外商,P,強制解約その他,6
663,2020/05/31,外商,Q,不良会員,3
664,2020/05/31,外商,W,カード不着,18
665,2020/05/31,外商,X,年会費未払い,14


In [97]:
#CSV出力
df_all.to_csv(data_dir /r"output"/'no_of_cancel_by_reason.csv', header=True ,index=None, encoding='shift-jis')

In [98]:
#入会年月別数(理由別)
q = """
        SELECT
        cp
        ,card
        ,membership_term
        ,count(no) as sum_no_of_account
        from 
            df_merge
        group by
            1,2,3
        """
df_shukei = sqldf(q, locals())
df_shukei.head()

,cp,card,membership_term,sum_no_of_account
0,2019/03/31,ゴールド,10年以上,14
1,2019/03/31,ゴールド,1年〜2年未満,44
2,2019/03/31,ゴールド,2年〜3年未満,35
3,2019/03/31,ゴールド,3ヶ月〜6ヶ月未満,5
4,2019/03/31,ゴールド,3ヶ月未満,2


In [99]:
#CSV出力
df_shukei.to_csv(data_dir /r"output"/'no_of_cancel_by_membership_term.csv', header=True ,index=None, encoding='shift-jis')

In [110]:
#年会費徴取タイミング別
q = """
        SELECT
        cp
        ,card
        ,abs(dif_nenkaihi) as dif_fee_month
        ,count(no) as sum_no_of_account
        from 
            df_merge
        group by
            1,2,3
        """
df_shukei = sqldf(q, locals())
df_shukei

,cp,card,dif_fee_month,sum_no_of_account
0,2019/03/31,ゴールド,0,17
1,2019/03/31,ゴールド,1,52
2,2019/03/31,ゴールド,2,44
3,2019/03/31,ゴールド,3,24
4,2019/03/31,ゴールド,4,32
...,...,...,...,...
418,2020/05/31,外商,3,39
419,2020/05/31,外商,4,30
420,2020/05/31,外商,5,83
421,2020/05/31,外商,6,11


In [111]:
#CSV出力
df_shukei.to_csv(data_dir /r"output"/'no_of_cancel_by_nenkaihi.csv', header=True ,index=None, encoding='shift-jis')

In [112]:
#メール有無別
q = """
        SELECT
        cp
        ,card
        ,email_f
        ,web_f
        ,count(no) as sum_no_of_account
        from 
            df_merge
        group by
            1,2,3,4
        """
df_shukei = sqldf(q, locals())
df_shukei.head()

,cp,card,email_f,web_f,sum_no_of_account
0,2019/03/31,ゴールド,0,0,130
1,2019/03/31,ゴールド,0,1,16
2,2019/03/31,ゴールド,1,0,45
3,2019/03/31,ゴールド,1,1,38
4,2019/03/31,一般,0,0,9303
...,...,...,...,...,...
170,2020/05/31,一般,1,1,1264
171,2020/05/31,外商,0,0,354
172,2020/05/31,外商,0,1,1
173,2020/05/31,外商,1,0,29


In [113]:
#CSV出力
df_shukei.to_csv(data_dir /r"output"/'no_of_cancel_by_email_f.csv', header=True ,index=None, encoding='shift-jis')

In [115]:
#最終利用月数別
q = """
        SELECT
        cp
        ,card
        ,case 
        when last_use_date in ("2050/12/31") then '09_non_active'
        when mob_last_use <= 3 then '01_<=3month'
        when mob_last_use <= 6 then '02_<=6month'
        when mob_last_use <= 9 then '03_<=9month'
        when mob_last_use <= 12 then '04_<=12month'
        else '05_12month<'
        end as last_use
        ,count(no) as sum_no_of_accoun
        from 
            df_merge
        group by
            1,2,3
        """
df_shukei = sqldf(q, locals())
df_shukei.head()

,cp,card,last_use,sum_no_of_accoun
0,2019/03/31,ゴールド,01_<=3month,127
1,2019/03/31,ゴールド,02_<=6month,27
2,2019/03/31,ゴールド,03_<=9month,13
3,2019/03/31,ゴールド,04_<=12month,9
4,2019/03/31,ゴールド,05_12month<,35


In [117]:
#CSV出力
df_shukei.to_csv(data_dir /r"output"/'no_of_cancel_by_last_use.csv', header=True ,index=None, encoding='shift-jis')